# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [23]:
from data_loader import MyDataLoader
%load_ext autoreload 
%autoreload 2
dl = MyDataLoader("combine2")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
dl.normalize_train_data(norm_size=(250,250))

In [25]:
dl.make_colorization_data()

In [26]:
import keras.backend as K
import tensorflow as tf
from keras.layers import Input, Conv2D, BatchNormalization, UpSampling2D
from keras.models import Model
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

l2_reg = l2(1e-3)

img_rows = 250
img_cols = 250
kernel = 3
num_classes = 3

def build_model():
    input_tensor = Input(shape=(img_rows, img_cols, 1))
    x = Conv2D(64, (kernel, kernel), activation='relu',padding = 'same', name='conv1_1', kernel_initializer="he_normal",
               kernel_regularizer=l2_reg)(input_tensor)
    x = Conv2D(64, (kernel, kernel), activation='relu', padding = 'same', name='conv1_2', kernel_initializer="he_normal",
               kernel_regularizer=l2_reg)(x)
    x = BatchNormalization()(x)


    outputs = Conv2D(num_classes, (1, 1), activation='softmax', padding='same', name='pred')(x)

    model = Model(inputs=input_tensor, outputs=outputs, name="ColorNet")
    return model

model = build_model()

model.compile(loss='mse',
              optimizer='Adam')

In [31]:
try:
    del X
    del y

except:
    pass

X, y = dl.get_colorization_data()

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [30]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [29]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data=(X_val, y_val))

Epoch 1/100
255/255 [==============================] - 38s 116ms/step - loss: 14297.9971 - val_loss: 14312.2051
Epoch 2/100
255/255 [==============================] - 26s 101ms/step - loss: 14296.5234 - val_loss: 14312.0068
Epoch 3/100
255/255 [==============================] - 26s 101ms/step - loss: 14296.3994 - val_loss: 14311.9043
Epoch 4/100
255/255 [==============================] - 26s 100ms/step - loss: 14296.3535 - val_loss: 14311.9160
Epoch 5/100
255/255 [==============================] - 26s 103ms/step - loss: 14296.3174 - val_loss: 14311.8281
Epoch 6/100
255/255 [==============================] - 25s 98ms/step - loss: 14296.3008 - val_loss: 14311.9141
Epoch 7/100
255/255 [==============================] - 23s 92ms/step - loss: 14296.3037 - val_loss: 14311.8027
Epoch 8/100
255/255 [==============================] - 23s 87ms/step - loss: 14296.2959 - val_loss: 14311.7744
Epoch 9/100
255/255 [==============================] - 22s 87ms/step - loss: 14296.3125 - val_loss: 14311.9

KeyboardInterrupt: 

In [18]:
from PIL import Image
import numpy as np
from sys import getsizeof

print(getsizeof(X))

2909000128


In [22]:
print(X.shape)
print(X[11001])


(11636, 500, 500)
[[184 184 184 ... 187 187 187]
 [184 184 184 ... 187 187 187]
 [184 184 184 ... 187 187 187]
 ...
 [169 167 166 ... 209 209 209]
 [171 169 167 ... 209 209 209]
 [172 170 168 ... 209 209 209]]


In [7]:


# print(X.shape)
# print(X_test.shape)
# im = Image.fromarray(X_train[0], mode="YCbCr")
# im.show()
# print(X_test[0])

(9262, 500, 500, 3)
(9262, 500, 500, 3)
[[[ 55  45  35]
  [ 56  46  36]
  [ 56  46  36]
  ...
  [ 47  43  31]
  [ 47  43  31]
  [ 47  43  31]]

 [[ 52  42  32]
  [ 53  43  33]
  [ 54  44  34]
  ...
  [ 52  48  36]
  [ 52  48  36]
  [ 52  48  36]]

 [[ 49  39  29]
  [ 50  40  30]
  [ 51  41  31]
  ...
  [ 55  51  39]
  [ 55  51  39]
  [ 55  51  39]]

 ...

 [[146 106 107]
  [146 106 107]
  [145 105 105]
  ...
  [165 158 166]
  [165 158 166]
  [165 158 166]]

 [[158 118 119]
  [158 118 119]
  [157 117 117]
  ...
  [175 168 176]
  [175 168 176]
  [175 168 176]]

 [[165 125 126]
  [164 124 125]
  [164 124 124]
  ...
  [180 173 181]
  [180 173 181]
  [180 173 181]]]


In [8]:
from PIL import Image
import numpy as np

img = Image.open("test.jpg").convert('L')
img = img.resize((500,500))
img.show()
arr = np.array(img)

In [18]:
from PIL import Image
import numpy as np

img = Image.open("bla.jpg")
img = img.resize((500,500))
x=np.ndarray((500,500,3), dtype=np.short)
img0 = Image.fromarray(x)

TypeError: Cannot handle this data type: (1, 1, 3), <i2